# Faceoff - AI Face Swap

Follow my Instagram: https://www.instagram.com/xsocialandapps/

Portable tools: https://buymeacoffee.com/socialadapps

---

## Setup Instructions
1. Run the **Setup** cell below (this will take several minutes)
2. Once complete, run the **Main Code** cell to start the application

In [ ]:
# ============================================
# FACEOFF SETUP - Complete Installation
# ============================================

print("📦 Step 1: Cloning repository...")
import os

# Detect environment and set appropriate working directory
if os.path.exists('/content'):
    # Google Colab
    work_dir = '/content'
else:
    # Modal or other environments - use current directory
    work_dir = os.getcwd()

print(f"Working in: {work_dir}")

# Clone repository if it doesn't exist
if not os.path.exists(os.path.join(work_dir, 'faceoff')):
    %cd {work_dir}
    !git clone https://github.com/harry2141985/faceoff.git
else:
    print("  ℹ️  Repository already exists, skipping clone")

# Change to faceoff directory
faceoff_dir = os.path.join(work_dir, 'faceoff')
%cd {faceoff_dir}
print(f"  ✓ Working directory: {os.getcwd()}")

print("\n📦 Step 2: Installing PyTorch and requirements...")
!pip install torch==2.3.1+cu121 torchvision==0.18.1+cu121 torchaudio==2.3.1+cu121 --index-url https://download.pytorch.org/whl/cu121
!pip install -r requirements.txt

print("\n📦 Step 3: Installing system dependencies (ffmpeg, libmagic, cuDNN)...")
!apt-get update -qq
!apt-get install -y libglib2.0-0 libgl1 libmagic1 ffmpeg

# Install cuDNN 9 for ONNX Runtime
print("  → Installing cuDNN 9...")
!apt-get install -y libcudnn9-cuda-12

print("\n📦 Step 4: Downloading models...")

# Download inswapper model with validation
inswapper_path = os.path.join(faceoff_dir, "inswapper_128.onnx")
expected_size = 540000000  # ~540MB

if os.path.exists(inswapper_path):
    file_size = os.path.getsize(inswapper_path)
    if file_size < expected_size * 0.9:  # Allow 10% variance
        print(f"  ⚠️  inswapper_128.onnx appears corrupted (size: {file_size} bytes), re-downloading...")
        os.remove(inswapper_path)
    else:
        print(f"  ℹ️  inswapper_128.onnx already exists ({file_size} bytes)")

if not os.path.exists(inswapper_path):
    print("  → Downloading inswapper_128.onnx...")
    !wget -O {inswapper_path} https://huggingface.co/thebiglaskowski/inswapper_128.onnx/resolve/main/inswapper_128.onnx
    file_size = os.path.getsize(inswapper_path)
    print(f"  ✓ Downloaded inswapper_128.onnx ({file_size} bytes)")

# Download buffalo_l model
buffalo_dir = os.path.join(faceoff_dir, "models/buffalo_l")
if not os.path.exists(os.path.join(buffalo_dir, "1k3d68.onnx")):
    print("  → Downloading buffalo_l model...")
    !mkdir -p {buffalo_dir}
    !wget -q -O {buffalo_dir}/buffalo_l.zip https://github.com/deepinsight/insightface/releases/download/v0.7/buffalo_l.zip
    !unzip -q -o {buffalo_dir}/buffalo_l.zip -d {buffalo_dir}
    !rm {buffalo_dir}/buffalo_l.zip
    print("  ✓ Downloaded buffalo_l model")
else:
    print("  ℹ️  buffalo_l model already exists")

# Download GFPGAN model
model_folder = os.path.join(faceoff_dir, "models/gfpgan")
model_path = os.path.join(model_folder, "GFPGANv1.4.pth")
os.makedirs(model_folder, exist_ok=True)

if not os.path.exists(model_path):
    print("  → Downloading GFPGAN model...")
    import gdown
    url = "https://huggingface.co/gmk123/GFPGAN/resolve/main/GFPGANv1.4.pth"
    gdown.download(url, model_path, quiet=False)
    print(f"  ✓ Model downloaded to: {model_path}")
else:
    print(f"  ℹ️  GFPGAN model already exists at: {model_path}")

print("\n📦 Step 5: Applying compatibility patch...")
import importlib.util

try:
    basicsr_spec = importlib.util.find_spec("basicsr")
    if basicsr_spec is None or basicsr_spec.origin is None:
        print("  ⚠️  basicsr package not found. Patch will be skipped.")
    else:
        basicsr_path = os.path.dirname(basicsr_spec.origin)
        file_path = os.path.join(basicsr_path, "data", "degradations.py")
        
        if not os.path.exists(file_path):
            print(f"  ℹ️  Patch file not found (may not be needed): {file_path}")
        else:
            with open(file_path, "r") as f:
                content = f.read()
            
            # Only patch if needed
            if "from torchvision.transforms.functional_tensor import rgb_to_grayscale" in content:
                patched = content.replace(
                    "from torchvision.transforms.functional_tensor import rgb_to_grayscale",
                    "from torchvision.transforms.functional import rgb_to_grayscale"
                )
                
                with open(file_path, "w") as f:
                    f.write(patched)
                
                print("  ✓ Patch applied successfully!")
                print(f"  📁 Patched file at: {file_path}")
            else:
                print("  ℹ️  Patch not needed (already fixed or different version)")
except Exception as e:
    print(f"  ⚠️  Error applying patch: {e}")
    print("  Continuing anyway - the error may not affect functionality.")

print("\n✅ Setup complete! Ready to run main.py")
print("\n⚠️  NOTE: After setup, you may need to restart the kernel for cuDNN changes to take effect.")

# Run Application

Run the cell below to start Faceoff. This will launch the face swap interface.

In [ ]:
import os

# Detect environment and navigate to faceoff directory
if os.path.exists('/content/faceoff'):
    work_dir = '/content/faceoff'
elif os.path.exists('faceoff'):
    work_dir = os.path.join(os.getcwd(), 'faceoff')
else:
    # Already in faceoff directory
    work_dir = os.getcwd()

%cd {work_dir}
print(f"Running from: {os.getcwd()}")
print("=" * 50)

!python main.py